# Performance optimization overview

This tutorial illustrates the performance optimizations applied to the code generated by an `Operator`. As we shall see, most optimizations are automatically applied as they're known to systematically improve performance. Others, whose impact varies across different `Operator`s, are instead to be enabled through specific options.

An Operator has several preset **optimization levels**; the fundamental ones are `noop` and `advanced`. With `noop`, no performance optimizations are introduced by the compiler. With `advanced`, several flop-reducing and data locality _optimization passes_ are performed. Examples of flop-reducing optimization passes are common sub-expressions elimination and factorization; examples of data locality optimization passes are loop fusion and cache blocking. SIMD vectorization, via compiler auto-vectorization, is enforced through OpenMP pragmas.

An optimization pass may provide knobs, or **options**, for fine-grained tuning. As explained in the next sections, some of these options are given at compile-time, others at run-time.

**\*\* Remark \*\***

Parallelism -- both shared-memory (e.g., OpenMP) and distributed-memory (MPI) -- is _by default disabled_ and is _not_ controlled via the optimization level. In this tutorial we will also show how to enable OpenMP parallelism (you'll see it's trivial!). A mini-guide about parallelism in Devito and related aspects is also available [here](https://github.com/devitocodes/devito/tree/master/benchmarks/user#a-step-back-configuring-your-machine-for-reliable-benchmarking). 

**\*\*\*\***

## API

The optimization level may be changed in various ways:

* globally, through the `DEVITO_OPT` environment variable. For example, to disable all optimizations on all `Operator`s, one should run with

```
DEVITO_OPT=noop python ...
```

* programmatically, adding the following lines to a program

```
from devito import configuration
configuration['opt'] = 'noop'
```

* locally, as an `Operator` argument

```
Operator(..., opt='noop')
```

Local takes precedence over programmatic, and programmatic takes precedence over global.

The optimization options, instead, may only be changed locally. The syntax to specify an option is

```
Operator(..., opt=('advanced', {<optimization options>})
```

A concrete example (you can ignore the meaning for now) is

```
Operator(..., opt=('advanced', {'blocklevels': 2})
```

That is, options are to be specified _together with_ the optimization level.

## Default values

By default, all `Operator`s are run with the optimization level set to `advanced` and with all options disabled. So this

```
Operator(Eq(...))
```

is equivalent to

```
Operator(Eq(...), opt='advanced')
```

and obviously also to

```
Operator(Eq(...), opt=('advanced', {}))
```

In virtually all scenarios, regardless of application and underlying architecture, this ensures very good performance -- but not necessarily the very best.

## Utilities

The following functions will be used throughout the notebook for various purposes.

In [ ]:
#TODO: George -- I guess we could (and should!) move this under examples/performance/utils.py ... 
def _unidiff_output(expected, actual):
    """
    Return a string containing the unified diff of two multiline strings.
    """
    import difflib
    expected=expected.splitlines(1)
    actual=actual.splitlines(1)

    diff=difflib.unified_diff(expected, actual)

    return ''.join(diff)

def _print_kernel(op):
    """
    Print the core part of an Operator used in this notebook.
    This is less verbose than printing the whole Operator.
    """
    if op._func_table:
        for i in op._func_table.values():
            print(i.root)
    else:
        print(op.body[-1])

## Running example

Throughout the notebook we will generate `Operator`s for the following time-marching `Eq`.

In [ ]:
from sympy import sin
from devito import Eq, Grid, Operator, Function, TimeFunction

grid = Grid(shape=(80, 80, 80))

f = Function(name='f', grid=grid)
u = TimeFunction(name='u', grid=grid, space_order=2)

eq = Eq(u.forward, sin(f)*u.dy.dy)

Despite its simplicity, this `Eq` is all we need to showcase the key components of the Devito optimization engine.

## OpenMP Parallelism

There are several ways to enable OpenMP parallelism. The one we use here consists of supplying an **option** to an `Operator`. The next cell illustrates the difference between two `Operator`s generated with the `noop` optimization level, but with OpenMP enabled on the latter `Operator`. 

In [ ]:
# NOTE: we only need this cell for Continuous Integration, which runs with OpenMP enabled
from devito import configuration
configuration['language'] = 'C'

In [ ]:
op0 = Operator(eq, opt=('noop'))
op0_omp = Operator(eq, opt=('noop', {'openmp': True}))

# print(op0)
# print(_unidiff_output(str(op0), str(op0_omp)))  # Uncomment to print out the diff only
_print_kernel(op0_omp)

The OpenMP-ized `op0_omp` `Operator`:

 - includes the necessary header file `#include "omp.h"`
 - the `#pragma omp parallel num_threads(nthreads)` directive
 - the `#pragma omp for collapse(...) schedule(dynamic,1)` directive
 
More complex `Operator`s will have more directives, more types of directives, different iteration scheduling strategies based on heuristics and empirical tuning (e.g., `static` instead of `dynamic`), etc.

We note how the OpenMP optimization pass also introduces a new symbol, `nthreads`. This allows users to explicitly control the number of threads with which an `Operator` is run.

In [ ]:
op0_omp.apply(time_M=0)  # Picks up `nthreads` from the standard environment variable OMP_NUM_THREADS
op0_omp.apply(time_M=0, nthreads=2)  # Runs with 2 threads per parallel loop

## The `advanced` mode

As already explained, `advanced` is the default optimization level in Devito. This mode performs several compilation passes to optimize the `Operator` for computation (number of flops), working set size, and data locality.

In the next paragraphs we dissect the `advanced` mode and analyze, one by one, some of its key passes.

### Loop blocking

The next cell creates a new `Operator` that adds loop blocking to what we had in `op0_omp`. 

In [ ]:
op1_omp = Operator(eq, opt=('blocking', {'openmp': True}))
print(op1_omp)

**\*\* Remark \*\***

`'blocking'` is **not** an optimization level -- it rather identifies a very specific compilation pass. In other words, the `advanced` mode represents a set of passes, and `blocking` is one such pass.

**\*\*\*\***

The `blocking` pass creates additional loops over blocks. In this simple `Operator` there is just one loop nest, so only a pair of additional loops are created. In more complex `Operator`s, several loop nests may individually be blocked, whereas others may be left unblocked -- this is decided by the Devito compiler according to certain heuristics. The size of a block is represented by the symbols `x0_blk0_size` and `y0_blk0_size`, which are runtime parameters akin to `nthreads`. 

By default, Devito applies 2D blocking and sets the default block shape to 8x8. There are two ways to set a different block shape:

* passing an explicit value. In the example below we would run with a 24x8 block shape

```
op1_omp.apply(..., x0_blk0_size=24)
```

* letting the autotuner run for a while, looking for a better block shape. There are several autotuning modes. A short summary is available [here](https://github.com/devitocodes/devito/wiki/FAQ#devito_autotuning)

```
op1_omp.apply(..., autotune='aggressive')
```

Loop blocking also provides two optimization options:

* `blockinner={False, True}` -- to enable 3D (or any nD, n>2) blocking
* `blocklevels={int}` -- to enable hierarchical blocking, to exploit the cache hierarchy 

In the example below, we construct an `Operator` featuring six-dimensional loop blocking, where the first three loops represent outer blocks, whereas the second three loops represent inner blocks within an outer block.

In [ ]:
op1_omp_6D = Operator(eq, opt=('blocking', {'blockinner': True, 'blocklevels': 2, 'openmp': True}))
print(op1_omp_6D)

### SIMD vectorization

Devito enforces SIMD vectorization through OpenMP pragmas.

In [ ]:
op2_omp = Operator(eq, opt=('blocking', 'simd', {'openmp': True}))
# print(op2_omp)  # Uncomment to see the generated code
# print(_unidiff_output(str(op1_omp), str(op2_omp)))  # Uncomment to print out the diff only

### Code motion

The `advanced` mode performs a code motion pass. In explicit PDE solvers, this is most commonly used to lift expensive time-invariant sub-expressions out of the inner loops. The pass is however quite general in that it is not restricted to the concept of time-invariance -- any sub-expression invariant with respect to a subset of `Dimension`s is a code motion candidate. In the example below, we see that `sin(f)` gets hoisted out of the inner loops since it is determined to be an expensive invariant sub-expression. In other words, the compiler trades the redundant computation of `sin(f)` for additional storage (the `r1[...]` array).

In [ ]:
op3_omp = Operator(eq, opt=('lift', {'openmp': True}))
_print_kernel(op3_omp)

### Basic flop-reducing transformations

Among the simpler flop-reducing transformations applied by the `advanved` mode we find:

* "classic" common sub-expressions elimination (CSE),
* factorization,
* optimization of powers

The cell below shows how the computation of `u` changes by incrementally applying these three passes. First of all, we observe how the symbolic spacing `h_y` gets assigned to a temporary, `r0`, due to appearing in several sub-expressions. This is the effect of CSE.

In [ ]:
op4_omp = Operator(eq, opt=('cse', {'openmp': True}))
print(_unidiff_output(str(op0_omp), str(op4_omp)))

Factorization causes the collection of `r0` in two different sub-expressions. In larger, more complex examples, the impact of factorization is typically much more pronounced.

In [ ]:
op5_omp = Operator(eq, opt=('cse', 'factorize', {'openmp': True}))
print(_unidiff_output(str(op5_omp), str(op4_omp)))

There are no powers in this example, so the `opt-pows` pass has no effect.

In [ ]:
op6_omp = Operator(eq, opt=('cse', 'factorize', 'opt-pows', {'openmp': True}))
print(_unidiff_output(str(op6_omp), str(op5_omp)))

### Cross-iteration redundancy elimination (CIRE)

This is perhaps the most advanced among the optimization passes in Devito. CIRE [1] searches for redundancies across consecutive loop iterations. These are often induced by a mixture of nested, high-order, partial derivatives. The underlying idea is very simple. Consider:

```
r0 = a[i-1] + a[i] + a[i+1]
```

at `i=1`, we have

```
r0 = a[0] + a[1] + a[2]
```

at `i=2`, we have

```
r0 = a[1] + a[2] + a[3]
```

So the sub-expression `a[1] + a[2]` is computed twice, by two consecutive iterations.
What makes CIRE complicated is the generalization to arbitrary expressions, the presence of multiple dimensions, the scheduling strategy due to the trade-off between redundant compute and working set, the co-existance with other optimizations (e.g., blocking, vectorization), and much more. All these aspects won't be treated here. What instead we will show is the effect of CIRE in our running example and the optimization options at our disposal to drive the detection and scheduling of the captured redundancies.

In our running example, some cross-iteration redundancies are induced by the nested first-order derivatives along `y`. As we see below, these redundancies are captured and assigned to the two-dimensional temporary `r3`.

Note: the name `cire-sops` means "Apply CIRE to sum-of-product expressions". A sum-of-product is what taking a derivative via finite differences produces.

In [ ]:
op7_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
_print_kernel(op7_omp)
# print(_unidiff_output(str(op7_omp), str(op0_omp)))  # Uncomment to print out the diff only

We note that since there are no redundancies along `x`, the compiler is smart to figure out that `r3` and `u` can safely be computed within the same `x` loop. This is nice -- not only is the reuse distance decreased, but also a grid-sized temporary is avoided.

#### The `min-storage` option

Let's now consider a variant of our running example

In [ ]:
eq = Eq(u.forward, sin(f)*(u.dy.dy + u.dx.dx))
op7_b0_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
_print_kernel(op7_b0_omp)

As expected, there are now two temporaries, one stemming from `u.dy.dy` and the other from `u.dx.dx`.  A key difference here is that both are grid-sized temporaries. This might seem odd at first -- why should the `u.dy.dy` temporary, that is `r6`, now be a three-dimensional temporary when we know already it could be a two-dimensional temporary? This is merely a compiler heuristic: by adding an extra dimension to `r6`, both temporaries can be scheduled within the same loop nest, thus augmenting data reuse and potentially enabling further cross-expression optimizations. We can disable this heuristic through the `min-storage` option.

In [ ]:
op7_b1_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'min-storage': True}))
_print_kernel(op7_b1_omp)

#### The `cire-mincost-sops` option

So far so good -- we've seen that Devito can capture and schedule cross-iteration redundancies. But what if we actually do _not_ want certain redundancies to be captured? There are a few reasons we may like that option, for example we're allocating too much extra memory for the tensor temporaries, and we rather prefer to avoid that. For this, we can tell Devito what the minimum cost of an sub-expression should be in order to be a CIRE candidate. The _cost_ is an integer number based on the operation count and the type of operations:

* A basic arithmetic operation such as `+` and `*` has a cost of 1.
* A `/` has a cost of 25.
* A power with non-integer exponent has a cost of 50.
* A power with non-negative integer exponent `n` has a cost of `n-1` (i.e., the number of `*` it will be converted into).
* A trascendental function (`sin`, `cos`, etc.) has a cost of 50.

The `cire-mincost-sops` option can be used to control the minimum cost of CIRE candidates.

In [ ]:
eq = Eq(u.forward, sin(f)*u.dy.dy)  # Back to original running example
op8_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'cire-mincost-sops': 107}))
_print_kernel(op8_omp)

We observe how setting `cire-min-cost=107` makes the tensor temporary produced by `op7_omp` disappear. 106 is indeed the cost of the targeted sub-expression

```
-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y
```

* One `-` => 1
* One `+` => 1
* Two `/h_y` => 50
* The sub-expression appears `n=2` times with shifted indices (precisely what CIRE looks for), so total cost to be multiplied by `n`.

Note: the cost of integer arithmetic for array indexing is deliberately ignored.

#### The `cire-repeats-sops` option

Higher order derivatives -- typically higher than order two -- may lead to proportionately larger sum-of-products. Consider the following variant of the running example

In [ ]:
eq = Eq(u.forward, sin(f)*u.dy.dy.dy.dy)
op9_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
_print_kernel(op9_omp)

At first glance, it's the same story as before, with a two-dimensional tensor temporary extracted and computed in a separate loop nest. By paying closer attention, however, we may note that there are now four occurrences of `r3` in the expression computing `u`, while before, in `op7_omp` for instance, we had only two. Devito seeks for a compromise between compilation speed and run-time performance, so it runs by default searching for redundancies across sum-of-products of `depth=5` or lower. `depth=5` sum-of-products are the typical by-product of second-order derivatives, which is what many PDEs have. There are other reasons behind the default value, which for brevity aren't discussed here. To search for deeper sum-of-products (and, therefore, for redundancies induced by higher order derivatives), one shall use the `cire-repeats-sops` option.

In [ ]:
op10_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'cire-repeats-sops': 7}))
_print_kernel(op10_omp)

#### The `cire-mincost-inv` and `cire-repeats-inv` options

Akin to sum-of-products, cross-iteration redundancies may be searched across dimension-invariant sub-expressions, typically time-invariants. So, analogously to what we've seen before:

In [ ]:
eq = Eq(u.forward, sin(f)*u.dy.dy)  # Back to original running example
op11_omp = Operator(eq, opt=('lift', {'openmp': True}))
# _print_kernel(op11_omp)  # Uncomment to see the generated code

For convenience, the `lift` pass triggers CIRE for dimension-invariant sub-expressions. As seen before, this leads to the `r1` temporary. By setting a larger value for `cire-mincost-inv`, we avoid a grid-size temporary to be allocated, in exchange for a trascendental function, `sin`, to be computed at each iteration

In [ ]:
op12_omp = Operator(eq, opt=('lift', {'openmp': True, 'cire-mincost-inv': 51}))
_print_kernel(op12_omp)

#### All optimizations together

The `advanced` mode triggers all of the passes we've seen so far, and in fact many more -- some are visible printing `op` (e.g., the addition of denormal-numbers related macros), others aren't (e.g., all of the MPI-related optimizations) and they will be covered in another tutorial. 

Obviously, all of the optimization options (e.g., `min-cost`, `cire-mincost-sops`, `blocklevels`) are applicable and composable in any arbitrary way.

In [ ]:
op13_omp = Operator(eq, openmp=True)
print(op13_omp)
# op14_omp = Operator(eq, opt=('advanced', {'min-storage': True}))

# References

[1] Fabio Luporini, Mathias Louboutin, Michael Lange, Navjot Kukreja, Philipp Witte, Jan Hückelheim, Charles Yount, Paul H. J. Kelly, Felix J. Herrmann, and Gerard J. Gorman. 2020. Architecture and Performance of Devito, a System for Automated Stencil Computation. ACM Trans. Math. Softw. 46, 1, Article 6 (April 2020), 28 pages. DOI:https://doi.org/10.1145/3374916